In [7]:
import cv2
cap = cv2.VideoCapture(r"C:\Users\ercih\Desktop\aspava-yildiz\python\images_data\video\Peter v1.0.mp4")

In [8]:
from pydoc import apropos
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import torchvision
import numpy as np

import cv2
import random
import warnings
import argparse
import os

warnings.filterwarnings('ignore')

# load model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True).cuda()
# set to evaluation mode
model.eval() 

# load COCO category names
COCO_CLASS_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]



def get_coloured_mask(mask):
  colours = [[0, 255, 0],[0, 0, 255],[255, 0, 0],[0, 255, 255],[255, 255, 0],[255, 0, 255],[80, 70, 180],[250, 80, 190],[245, 145, 50],[70, 150, 250],[50, 190, 190]]
  r = np.zeros_like(mask).astype(np.uint8)
  g = np.zeros_like(mask).astype(np.uint8)
  b = np.zeros_like(mask).astype(np.uint8)
  r[mask == 1], g[mask == 1], b[mask == 1] = colours[2]
  coloured_mask = np.stack([r, g, b], axis=2)
  return coloured_mask



def get_prediction(frame, confidence):
  img_pil = Image.fromarray(frame)
#   img_return = img.resize((width, height), Image.BILINEAR)
  transform = T.Compose([T.ToTensor()])
  img = transform(img_pil).cuda()
  pred = model([img])
  pred_score = list(pred[0]['scores'].cpu().detach().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x>confidence][-1]
  pred_class = [COCO_CLASS_NAMES[i] for i in list(pred[0]['labels'].cpu().detach().numpy())]
  dog_index = pred_class.index("dog")
  masks = (pred[0]['masks']>0.5).squeeze().detach().cpu().numpy()
  
  masks = masks[dog_index]
  
  pred_class = pred_class[:pred_t+1]
  return masks, pred_class, frame




def segment_instance(frame, confidence=0.5):
    masks, pred_cls, img = get_prediction(frame, confidence)
    rgb_mask = get_coloured_mask(masks)
    print(np.shape(img), np.shape(rgb_mask), np.shape(masks))
    img = cv2.addWeighted(img, 1, rgb_mask, 0.5, 0)
    return img


In [9]:
# We need to set resolutions.
# so, convert them from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)

# Below VideoWriter object will create
# a frame of above defined The output
# is stored in 'filename.avi' file.
result = cv2.VideoWriter('video_segmented.avi',
						cv2.VideoWriter_fourcc(*'MJPG'),
						30, size)
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Display the resulting frame
    # cv2.imshow('Frame',segment_instance(frame))
    result.write(segment_instance(frame))

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

cap.release()
result.release()

(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720, 1280)
(720, 1280, 3) (720, 1280, 3) (720